In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define a simple Variational Autoencoder (VAE) as the generative model
class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim * 2)  # Two times latent_dim for mean and log-variance
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim),
            nn.Sigmoid()
        )
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def forward(self, x):
        x = self.encoder(x)
        mu, log_var = x[:, :latent_dim], x[:, latent_dim:]
        z = self.reparameterize(mu, log_var)
        reconstructed = self.decoder(z)
        return reconstructed, mu, log_var

# Generate synthetic data for demonstration (replace with your actual dataset)
num_samples = 1000
data_dim = 20
data = torch.tensor(np.random.randint(2, size=(num_samples, data_dim)), dtype=torch.float32)

print(data)

# Initialize the VAE model
input_dim = data_dim
hidden_dim = 64
latent_dim = 16
vae = VAE(input_dim, hidden_dim, latent_dim)

# Define an adversary model (a simple feedforward neural network)
class Adversary(nn.Module):
    def __init__(self, input_dim):
        super(Adversary, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.fc(x)

# Train the VAE model (replace with your actual training process)

# Train the adversary model
adversary = Adversary(latent_dim)
optimizer = optim.Adam(adversary.parameters(), lr=0.001)
criterion = nn.BCELoss()

# Prepare target data for the membership inference attack
target_data_point = torch.tensor(np.random.randint(2, size=data_dim), dtype=torch.float32)

# Membership inference attack function
def membership_inference_attack(vae, adversary, target_data_point):
    # Encode the target data point using the VAE
    with torch.no_grad():
        target_data_point = target_data_point.unsqueeze(0)  # Add batch dimension
        reconstructed, mu, log_var = vae(target_data_point)
    
    # Use the adversary to predict membership
    prediction = adversary(mu)
    
    # If the prediction is close to 1, the target data point is likely a member
    if prediction.item() > 0.5:
        return "Member"
    else:
        return "Non-Member"

# Perform the membership inference attack
result = membership_inference_attack(vae, adversary, target_data_point)

# Output the result
print("Membership Inference Result:", result)

tensor([[1., 1., 0.,  ..., 1., 1., 1.],
        [1., 0., 1.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 1., 0., 1.],
        ...,
        [1., 0., 1.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [1., 1., 0.,  ..., 1., 0., 0.]])
Membership Inference Result: Member
